In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
# Specify the directory containing the file
directory = '/content/'

# List all files in the specified directory
for dirname, _, filenames in os.walk(directory):
    for filename in filenames:
        if filename == 'news.csv':
            print(os.path.join(dirname, filename))

In [3]:
# Change the path for Google Colab
source_file = 'D:/Downloads/news.csv'


In [4]:
# Load the dataset into a pandas dataframe.
df = pd.read_csv(source_file)

# Report the number of sentences.
print('Number of training sentences: {:,}\n'.format(df.shape[0]))

# Display 10 random rows from the data.
df.sample(10)

Number of training sentences: 6,335



,Unnamed: 0,title,text,label
5449,7250,Pa. lawmakers approve ban on naming officers i...,Print \nAs cities nationwide grapple with fata...,FAKE
772,3073,Booze Brands Polarize Just Like Politicians,A gay man is selling beer during the NBA playo...,REAL
3752,1961,Rubio looks to April 13 Miami launch,A verdict in 2017 could have sweeping conseque...,REAL
1035,9848,Our Debt to Paula Jones,"Fitzgerald Griffin Foundation October 29, 2016...",FAKE
1207,4407,Boehner 'baffled' by Israeli spying report,Washington (CNN) U.S. lawmakers and administra...,REAL
6020,2637,Eric Holder bids farewell to Justice Department,Hundreds of Justice Department staff and lawye...,REAL
4926,5930,What is the goal of the Progressive Movement: ...,pirate clip art free printable | Illustration ...,FAKE
5147,6070,Are you taking your iodine?,"Wed, 26 Oct 2016 08:04 UTC © periodictable.com...",FAKE
2733,8153,"Why the ECB is Going to Print, Print, PRINT!",by Yves Smith \nYves here. This article is a s...,FAKE
407,1784,What Is Hillary’s Greatest Accomplishment?,"“If you want to stump a Democrat, ask them to ...",REAL


In [5]:
text=df['text']
label = df['label']

In [6]:
print (text[1])

Google Pinterest Digg Linkedin Reddit Stumbleupon Print Delicious Pocket Tumblr 
There are two fundamental truths in this world: Paul Ryan desperately wants to be president. And Paul Ryan will never be president. Today proved it. 
In a particularly staggering example of political cowardice, Paul Ryan re-re-re-reversed course and announced that he was back on the Trump Train after all. This was an aboutface from where he was a few weeks ago. He had previously declared he would not be supporting or defending Trump after a tape was made public in which Trump bragged about assaulting women. Suddenly, Ryan was appearing at a pro-Trump rally and boldly declaring that he already sent in his vote to make him President of the United States. It was a surreal moment. The figurehead of the Republican Party dosed himself in gasoline, got up on a stage on a chilly afternoon in Wisconsin, and lit a match. . @SpeakerRyan says he voted for @realDonaldTrump : “Republicans, it is time to come home” https

Encoding Part


In [7]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
yt = le.fit_transform(label)
print(label)
print(yt)

0       FAKE
1       FAKE
2       REAL
3       FAKE
4       REAL
        ... 
6330    REAL
6331    FAKE
6332    FAKE
6333    REAL
6334    REAL
Name: label, Length: 6335, dtype: object
[0 0 1 ... 0 1 1]


EDA

In [8]:
#Missing values in training set
print(df.isnull().sum())

Unnamed: 0    0
title         0
text          0
label         0
dtype: int64


In [12]:

!pip install torch --timeout=120 --index-url https://pypi.org/simple



  Using cached torch-2.5.0-cp311-cp311-win_amd64.whl.metadata (28 kB)
  Using cached filelock-3.16.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2024.9.0-py3-none-any.whl.metadata (11 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ---------------------------------------- 0.0/203.1 MB ? eta -:--:--
   ---------------------------------------- 0.3/203.1 MB ? eta -:--:--
   ---------------------------------------- 0.8/203.1 MB 1.7 MB/s eta 0:02:01
   ---------------------------------------- 0.8/203.1 MB 1.7 MB/s eta 0:02:01
   ---------------------------------------- 1.0/203.1 MB 1.5 MB/s eta 0:02:17
   ---------------------------------------- 1.3/203.1 MB 1.3 MB/s eta 0:02:31
   ---------------------------------------- 1.3/203.1 MB 1.3 MB/s eta 0:02:31
   ---------------------------------------- 1.6/203.1 MB 1.0 MB/s eta 0:03:15
   ---------------------------------------- 1.6/203.1 MB 1.0 MB/

In [13]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


In [21]:
from transformers import BertTokenizer

print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)


  Using cached transformers-4.45.2-py3-none-any.whl.metadata (44 kB)
  Using cached huggingface_hub-0.26.0-py3-none-any.whl.metadata (13 kB)
  Using cached PyYAML-6.0.2-cp311-cp311-win_amd64.whl.metadata (2.1 kB)
  Using cached safetensors-0.4.5-cp311-none-win_amd64.whl.metadata (3.9 kB)
  Using cached tokenizers-0.20.1-cp311-none-win_amd64.whl.metadata (6.9 kB)
   ---------------------------------------- 0.0/9.9 MB ? eta -:--:--
   -- ------------------------------------- 0.5/9.9 MB 4.2 MB/s eta 0:00:03
   --- ------------------------------------ 0.8/9.9 MB 2.6 MB/s eta 0:00:04
   ----- ---------------------------------- 1.3/9.9 MB 2.2 MB/s eta 0:00:04
   ------- -------------------------------- 1.8/9.9 MB 2.1 MB/s eta 0:00:04
   --------- ------------------------------ 2.4/9.9 MB 2.2 MB/s eta 0:00:04
   ---------- ----------------------------- 2.6/9.9 MB 2.1 MB/s eta 0:00:04
   ------------ --------------------------- 3.1/9.9 MB 2.1 MB/s eta 0:00:04
   ------------- -----------------

C:\Users\Pulkit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading BERT tokenizer...


C:\Users\Pulkit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Pulkit\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [15]:
# Print the original sentence.
print(' Original: ', text[0])

 Original:  Daniel Greenfield, a Shillman Journalism Fellow at the Freedom Center, is a New York writer focusing on radical Islam. 
In the final stretch of the election, Hillary Rodham Clinton has gone to war with the FBI. 
The word “unprecedented” has been thrown around so often this election that it ought to be retired. But it’s still unprecedented for the nominee of a major political party to go war with the FBI. 
But that’s exactly what Hillary and her people have done. Coma patients just waking up now and watching an hour of CNN from their hospital beds would assume that FBI Director James Comey is Hillary’s opponent in this election. 
The FBI is under attack by everyone from Obama to CNN. Hillary’s people have circulated a letter attacking Comey. There are currently more media hit pieces lambasting him than targeting Trump. It wouldn’t be too surprising if the Clintons or their allies were to start running attack ads against the FBI. 
The FBI’s leadership is being warned that the

In [22]:
# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(text[0]))

Tokenized:  ['daniel', 'greenfield', ',', 'a', 'shi', '##ll', '##man', 'journalism', 'fellow', 'at', 'the', 'freedom', 'center', ',', 'is', 'a', 'new', 'york', 'writer', 'focusing', 'on', 'radical', 'islam', '.', 'in', 'the', 'final', 'stretch', 'of', 'the', 'election', ',', 'hillary', 'rod', '##ham', 'clinton', 'has', 'gone', 'to', 'war', 'with', 'the', 'fbi', '.', 'the', 'word', '“', 'unprecedented', '”', 'has', 'been', 'thrown', 'around', 'so', 'often', 'this', 'election', 'that', 'it', 'ought', 'to', 'be', 'retired', '.', 'but', 'it', '’', 's', 'still', 'unprecedented', 'for', 'the', 'nominee', 'of', 'a', 'major', 'political', 'party', 'to', 'go', 'war', 'with', 'the', 'fbi', '.', 'but', 'that', '’', 's', 'exactly', 'what', 'hillary', 'and', 'her', 'people', 'have', 'done', '.', 'coma', 'patients', 'just', 'waking', 'up', 'now', 'and', 'watching', 'an', 'hour', 'of', 'cnn', 'from', 'their', 'hospital', 'beds', 'would', 'assume', 'that', 'fbi', 'director', 'james', 'come', '##y', 'i

In [23]:
# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text[0])))

Token IDs:  [3817, 26713, 1010, 1037, 11895, 3363, 2386, 8083, 3507, 2012, 1996, 4071, 2415, 1010, 2003, 1037, 2047, 2259, 3213, 7995, 2006, 7490, 7025, 1012, 1999, 1996, 2345, 7683, 1997, 1996, 2602, 1010, 18520, 8473, 3511, 7207, 2038, 2908, 2000, 2162, 2007, 1996, 8495, 1012, 1996, 2773, 1523, 15741, 1524, 2038, 2042, 6908, 2105, 2061, 2411, 2023, 2602, 2008, 2009, 11276, 2000, 2022, 3394, 1012, 2021, 2009, 1521, 1055, 2145, 15741, 2005, 1996, 9773, 1997, 1037, 2350, 2576, 2283, 2000, 2175, 2162, 2007, 1996, 8495, 1012, 2021, 2008, 1521, 1055, 3599, 2054, 18520, 1998, 2014, 2111, 2031, 2589, 1012, 16571, 5022, 2074, 12447, 2039, 2085, 1998, 3666, 2019, 3178, 1997, 13229, 2013, 2037, 2902, 9705, 2052, 7868, 2008, 8495, 2472, 2508, 2272, 2100, 2003, 18520, 1521, 1055, 7116, 1999, 2023, 2602, 1012, 1996, 8495, 2003, 2104, 2886, 2011, 3071, 2013, 8112, 2000, 13229, 1012, 18520, 1521, 1055, 2111, 2031, 17640, 1037, 3661, 7866, 2272, 2100, 1012, 2045, 2024, 2747, 2062, 2865, 2718, 4109, 1

Encoding Tokens

In [24]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []

# For every sentence...
for sent in text:
    # `encode` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    encoded_sent = tokenizer.encode(
                        sent[:512],                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'

                        # This function also supports truncation and conversion
                        # to pytorch tensors, but we need to do padding, so we
                        # can't use these features :( .
                        #max_length = 128,          # Truncate all sentences.
                        #return_tensors = 'pt',     # Return pytorch tensors.
                   )

    # Add the encoded sentence to the list.
    input_ids.append(encoded_sent)

In [25]:
# Print sentence 0, now as a list of IDs.
print('Original: ', text[1])
print('Token IDs:', input_ids[1])

Original:  Google Pinterest Digg Linkedin Reddit Stumbleupon Print Delicious Pocket Tumblr 
There are two fundamental truths in this world: Paul Ryan desperately wants to be president. And Paul Ryan will never be president. Today proved it. 
In a particularly staggering example of political cowardice, Paul Ryan re-re-re-reversed course and announced that he was back on the Trump Train after all. This was an aboutface from where he was a few weeks ago. He had previously declared he would not be supporting or defending Trump after a tape was made public in which Trump bragged about assaulting women. Suddenly, Ryan was appearing at a pro-Trump rally and boldly declaring that he already sent in his vote to make him President of the United States. It was a surreal moment. The figurehead of the Republican Party dosed himself in gasoline, got up on a stage on a chilly afternoon in Wisconsin, and lit a match. . @SpeakerRyan says he voted for @realDonaldTrump : “Republicans, it is time to come 

Padding And Truncating

In [26]:
print('Max sentence length: ', max([len(sen) for sen in input_ids]))

Max sentence length:  289


In [27]:
# We'll borrow the `pad_sequences` utility function to do this.
from keras.preprocessing.sequence import pad_sequences

# Set the maximum sequence length.
# I've chosen 512 somewhat arbitrarily. It's slightly larger than the
# maximum training sentence length ...
MAX_LEN = 256

print('\nPadding/truncating all sentences to %d values...' % MAX_LEN)

print('\nPadding token: "{:}", ID: {:}'.format(tokenizer.pad_token, tokenizer.pad_token_id))

# Pad our input tokens with value 0.
# "post" indicates that we want to pad and truncate at the end of the sequence,
# as opposed to the beginning.
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long",
                          value=0, truncating="post", padding="post")

print('\nDone.')


Padding/truncating all sentences to 256 values...

Padding token: "[PAD]", ID: 0

Done.


Attention Masks

In [28]:
#Create attention masks
attention_masks = []

# For each sentence...
for sent in input_ids:

    # Create the attention mask.
    #   - If a token ID is 0, then it's padding, set the mask to 0.
    #   - If a token ID is > 0, then it's a real token, set the mask to 1.
    att_mask = [int(token_id > 0) for token_id in sent]

    # Store the attention mask for this sentence.
    attention_masks.append(att_mask)

Training & Validation Split

In [29]:
from sklearn.model_selection import train_test_split

# Use 90% for training and 10% for validation.
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, yt,
                                                            random_state=2018, test_size=0.1)
# Do the same for the masks.
train_masks, validation_masks, _, _ = train_test_split(attention_masks, yt,
                                             random_state=2018, test_size=0.1)

Converting to PyTorch Data Type

In [30]:
# Convert all inputs and labels into torch tensors, the required datatype
# for our model.
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)

train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)

train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

Creating DataLoader

In [31]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it
# here.
# For fine-tuning BERT on a specific task, the authors recommend a batch size of
# 16 or 32.

batch_size = 16

# Create the DataLoader for our training set.
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set.
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

Train Our Classification Model

In [32]:
torch.cuda.empty_cache()

In [33]:
from transformers import BertForSequenceClassification

# Load BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=2,
    output_attentions=False,
    output_hidden_states=False,
)

# Print the model architecture
print(model)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [34]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [35]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load the model
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=2,
    output_attentions=False,
    output_hidden_states=False,
)

# Create the optimizer
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Pulkit\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [36]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs (authors recommend between 2 and 4)
epochs = 4

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)
scheduler

TRAINING

In [37]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [38]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
import random
import numpy as np
import torch
import time
from transformers import BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup

# Set the seed for reproducibility
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# Hyperparameters
epochs = 3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize model, optimizer, and scheduler
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model.to(device)

optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

loss_values = []

for epoch_i in range(epochs):
    print(f"\n======== Epoch {epoch_i + 1} / {epochs} ========")
    print('Training...')
    t0 = time.time()
    total_loss = 0
    model.train()

    for step, batch in enumerate(train_dataloader):
        if step % 40 == 0 and step != 0:
            elapsed = format_time(time.time() - t0)
            print(f'  Batch {step:>5} of {len(train_dataloader):>5}. Elapsed: {elapsed}.')

        # Unpack the batch and move tensors to the correct device
        b_input_ids, b_input_mask, b_labels = batch[0].to(device), batch[1].to(device), batch[2].to(device)

        # Convert labels to torch.LongTensor
        b_labels = b_labels.long()

        # Reset the gradients
        model.zero_grad()

        # Forward pass
        outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs.loss

        # Accumulate the training loss
        total_loss += loss.item()

        # Backward pass
        loss.backward()

        # Clip the norm of the gradients to 1.0 to prevent exploding gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and learning rate
        optimizer.step()
        scheduler.step()

    # Calculate the average loss for the epoch
    avg_train_loss = total_loss / len(train_dataloader)
    loss_values.append(avg_train_loss)

    print(f"\n  Average training loss: {avg_train_loss:.2f}")
    print(f"  Training epoch took: {format_time(time.time() - t0)}")

    # Validation
    print("\nRunning Validation...")
    t0 = time.time()
    model.eval()

    eval_accuracy = 0
    nb_eval_steps = 0

    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        # Convert labels to torch.LongTensor
        b_labels = b_labels.long()

        # Disable gradient calculation for validation
        with torch.no_grad():
            outputs = model(b_input_ids, attention_mask=b_input_mask)
            logits = outputs.logits

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate accuracy
        eval_accuracy += flat_accuracy(logits, label_ids)
        nb_eval_steps += 1

    print(f"  Accuracy: {eval_accuracy / nb_eval_steps:.2f}")
    print(f"  Validation took: {format_time(time.time() - t0)}")

print("\nTraining complete!")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



======== Epoch 1 / 3 ========
Training...
  Batch    40 of   357. Elapsed: 0:08:02.
  Batch    80 of   357. Elapsed: 0:16:11.
  Batch   120 of   357. Elapsed: 0:23:57.
  Batch   160 of   357. Elapsed: 0:27:35.
  Batch   200 of   357. Elapsed: 0:31:13.
  Batch   240 of   357. Elapsed: 0:34:50.
  Batch   280 of   357. Elapsed: 0:38:27.
  Batch   320 of   357. Elapsed: 0:42:05.

  Average training loss: 0.22
  Training epoch took: 0:45:22

Running Validation...
  Accuracy: 0.95
  Validation took: 0:01:03

======== Epoch 2 / 3 ========
Training...
  Batch    40 of   357. Elapsed: 0:04:52.
